In [2]:
import os

print(os.path.abspath(os.curdir))
os.chdir("..")
print(os.path.abspath(os.curdir))

d:\Data Kuliah\Semester 4\Seminar 2\Definition Extraction\drone-definition\analysis
d:\Data Kuliah\Semester 4\Seminar 2\Definition Extraction\drone-definition


In [ ]:
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("DEEPSEEK_KEY")
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

with open("examples.json", "r", encoding="utf-8") as f:
    examples = json.load(f)

example_block = json.dumps(examples, indent=2)

prompt = f"""
You are a drone flight systems engineer with extensive experience in analyzing UAV flight logs and technical documentation. Your task is to help forensic investigators understand technical terms in flight logs by providing accurate, concise definitions.

Here are some examples of how you should define terms:
{example_block}

For any new term:
1. Only provide definitions based on standard drone manufacturer documentation and aviation protocols.
2. If a term could have multiple interpretations, focus only on its UAV-specific meaning.
3. Mark your confidence level as High, Medium, or Low. If you're uncertain, explicitly state "Definition requires verification".
4. Output the response strictly in a valid JSON list format as shown in the examples.

Please define the following term(s):
Sport Mode, Positioning Mode, Attitude Mode, Remote Controller, Interference, Gyroscope, LOS (Line Of Sight), Payload, RTH (Return to Home), VLOS (Visual Line of Sight), RTK (Real-Time Kinematic), Anti-Collision, Drone, Follow Me Mode, Gimbal, GPS, Hover, Home Point, IMU (Inertial Measurement Unit), Intelligent Flight Modes, No-Fly Zone, Obstacle Avoidance System, Propeller Guard, Remote ID, Waypoint, Propellers, Tripod mode, Pitch, Throttle, Yaw, Roll
"""
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": prompt},
    ],
    stream=False
)

In [ ]:
# Old constraints:
# 1. Only provide definitions based on standard drone manufacturer documentation and aviation protocols
# 2. If a term could have multiple interpretations, focus only on its UAV-specific meaning
# 3. Mark your confidence level as:
#    - High: Standard term with consistent definition across the industry
#    - Medium: Term may vary slightly between manufacturers
#    - Low: Definition is context-dependent or not standardized
# 4. If you're uncertain, explicitly state "Definition requires verification"

In [2]:
print(response.choices[0].message.content)

```json
[
    {
        "term": "Sport Mode",
        "definition": "A flight mode that prioritizes speed and agility by increasing the responsiveness of the drone's controls and often disabling or reducing certain obstacle avoidance and stabilization systems. This allows for faster flight but requires more pilot skill.",
        "confidence": "High"
    },
    {
        "term": "Positioning Mode",
        "definition": "The standard flight mode where the drone uses GPS and/or vision systems to automatically maintain a stable hover and precise position, making it easier for the pilot to control.",
        "confidence": "High"
    },
    {
        "term": "Attitude Mode",
        "definition": "A manual flight mode (often labeled 'ATTI') where the drone uses only its internal gyroscopes and accelerometers to maintain its attitude (level orientation) but does not use GPS or other sensors to hold its position. The drone will drift with wind or inertia.",
        "confidence": "High"
    }

In [5]:
import os
from datetime import datetime

curr_dt = datetime.now()
timestamp = int(round(curr_dt.timestamp()))
print("Current datetime: ", )
filename = '_'.join(str(curr_dt.date()).split(' '))
output_file = os.path.join('experiments', 'deepseek', f'{filename}_{timestamp}.txt')
print(output_file)
f = open(output_file, "w")
f.write(response.choices[0].message.content)
f.close()

Current datetime: 
experiments\deepseek\2025-12-25_1766661574.txt


In [ ]:
# load the old (no JSON constraints) definition
filename  = '2025-12-25'
timestamp = '1766661574'
definition_file = open(os.path.join('experiments', 'deepseek', f'{filename}_{timestamp}.txt'), "r")
definition_file.seek(0)
# readlines function
definition_dict = {
    'Term': [],
    'Definition': [],
    'Confidence': []
}
for line in definition_file.readlines():
    if line != '\n':
        if line.startswith('**'):
            key, value = line.replace('*', '').split(':')
            print(f'key: {key}')
            print(f'value: {value}')
            definition_dict[key].append(value.strip())
definition_file.close()

In [6]:
import json
import re
from typing import Any

def parse_llm_json_response(raw_text: str) -> Any:
    """
    Parse a JSON response produced by an LLM that may be wrapped
    in Markdown code fences (```json ... ```).

    Returns:
        Parsed Python object (dict or list)

    Raises:
        ValueError if no valid JSON can be extracted.
    """

    text = raw_text.strip()

    # 1. Remove Markdown code fences (``` or ```json)
    if text.startswith("```"):
        text = re.sub(r"^```(?:json)?\s*", "", text)
        text = re.sub(r"\s*```$", "", text)

    text = text.strip()

    # 2. Attempt direct JSON parse
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass

    # 3. Fallback: extract first JSON array or object
    json_pattern = re.compile(
        r"(\{.*\}|\[.*\])",
        re.DOTALL
    )

    match = json_pattern.search(text)
    if not match:
        raise ValueError("No JSON object or array found in LLM response.")

    json_str = match.group(1)

    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Extracted JSON is invalid: {e}")

In [8]:
# load the JSON definition
filename  = '2025-12-25'
timestamp = '1766661574'
with open(os.path.join('experiments', 'deepseek', f'{filename}_{timestamp}.txt'), "r", encoding="utf-8") as f:
    raw = f.read()
parsed = parse_llm_json_response(raw)

definition_dict = {
    'term': [],
    'definition': [],
    'confidence': []
}
for item in parsed:
    for key, value in item.items():
        definition_dict[key].append(value.strip())

In [ ]:
filename  = '2025-12-25'
timestamp = '1766661574'
definition_file = open(os.path.join('experiments', 'deepseek', f'{filename}_{timestamp}.txt'), "r")
definition_file.seek(0)
# readlines function
definition_dict = {
    'Term': [],
    'Definition': [],
    'Confidence': []
}
for line in definition_file.readlines():
    if line != '\n':
        if line.startswith('**'):
            key, value = line.replace('*', '').split(':')
            print(f'key: {key}')
            print(f'value: {value}')
            definition_dict[key].append(value.strip())
definition_file.close()

In [9]:
import pandas as pd

data = pd.DataFrame.from_dict(definition_dict)
data

,term,definition,confidence
0,Sport Mode,A flight mode that prioritizes speed and agili...,High
1,Positioning Mode,The standard flight mode where the drone uses ...,High
2,Attitude Mode,A manual flight mode (often labeled 'ATTI') wh...,High
3,Remote Controller,The handheld device used by the pilot to send ...,High
4,Interference,"Any unwanted disturbance that disrupts, degrad...",High
5,Gyroscope,A sensor within the drone's flight controller ...,High
6,LOS (Line Of Sight),"The direct, unobstructed visual path between t...",High
7,Payload,Any equipment carried by the drone that is not...,High
8,RTH (Return to Home),An automated safety feature where the drone na...,High
9,VLOS (Visual Line of Sight),A flight operation rule requiring the remote p...,High


In [10]:
data.to_excel(os.path.join('experiments', 'deepseek', f'{filename}_{timestamp}.xlsx'), index=False)